In [1]:
import os
import pathlib

project_root = pathlib.Path.cwd()

cache_dir = project_root / "models_cache"
cache_dir.mkdir(exist_ok=True)
os.environ['HF_HOME'] = str(cache_dir)
print(f"La variable de entorno HF_HOME se ha establecido en: {os.environ['HF_HOME']}")


La variable de entorno HF_HOME se ha establecido en: /Users/deimagjas/machinelearning/models_cache


In [9]:
import json
from typing import Dict, List, Tuple, Union

import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import load, generate
from mlx_lm.tuner import TrainingArgs, linear_to_lora_layers, train

In [3]:
from huggingface_hub import login

login()

## Carga de modelo desde HF

La razón por la que el código funciona con google/gemma-3-270m-it pero no con google/gemma-3-270m se debe a la diferencia entre los
  dos tipos de modelos:

   1. `google/gemma-3-270m-it`: El sufijo "-it" significa "Instruction Tuned" (ajustado para instrucciones). Este modelo ha sido
      específicamente entrenado para entender y seguir instrucciones en un formato de chat o de pregunta-respuesta. Su tokenizador
      incluye una "plantilla de chat" (chat template) que formatea la entrada de manera que el modelo la entienda.

   2. `google/gemma-3-270m`: Este es el modelo base. Es un modelo de lenguaje pre-entrenado que es bueno para predecir la siguiente
      palabra en un texto, pero no ha sido ajustado para seguir instrucciones o para conversar. Su tokenizador no tiene una plantilla de
      chat predeterminada.

In [4]:
model_path = "google/gemma-3-270m-it"
model, tokenizer = load(model_path)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
prompt = "generate an SQL query to find all users who registered in the last 30 days"
messages = [{"role": "user", "content": prompt}]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate(model, tokenizer, prompt=prompt, verbose=True)

```sql
SELECT *
FROM Users
WHERE Date('now') >= DATE('now', 30);
```

**Explanation:**

* **`SELECT *`**: This selects all columns from the `Users` table.
* **`FROM Users`**: This specifies the table to query from.
* **`WHERE Date('now') >= DATE('now', 30)`**: This is the filtering condition.
    * **`Date('now')`**: This function returns the current date and time.
    * **`DATE('now', 30)`**: This function returns the date 30 days ago.
    * **`>=`**: This operator checks if the date is greater than or equal to the current date.
    * **`DATE('now', 30)`**: This function returns the date 30 days ago.

This query will return a table with the following columns:

* **`SELECT *`**:  All columns from the `Users` table.
* **`FROM Users`**:  Specifies the table to retrieve data from.
* **`WHERE Date('now') >= DATE('now', 30)`**:  This filters the results to
Prompt: 26 tokens, 423.377 tokens-per-sec
Generation: 256 tokens, 118.615 tokens-per-sec
Peak memory: 0.566 GB


# Creando Adaptador

In [6]:
adapter_path = "adapters_gemma3"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

# Lora config

In [7]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 8,
        "scale": 20.0,
        "dropout": 0.0,
    },
}

In [8]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)

In [9]:
training_args = TrainingArgs(
    adapter_file=adapter_file_path,
    iters=200,
    steps_per_eval=50,
    grad_checkpoint=True,
)

# Parameters and adapter
La razón por la que ves 163,840 parámetros entrenables en lugar de los 270 millones del modelo completo es porque no estás
  re-entrenando el modelo entero. Estás utilizando una técnica de ajuste fino de alta eficiencia de parámetros (PEFT) llamada LoRA 
  (Low-Rank Adaptation).

  Así es como funciona en tu notebook:

   1. Congelar el modelo base: En la celda con el id: a3b86f5c, la primera línea es model.freeze(). Esto "congela" todos los 270 millones
      de parámetros del modelo Gemma, haciendo que no sean entrenables.

   2. Inyectar adaptadores LoRA: La siguiente línea, linear_to_lora_layers(...), añade pequeños "adaptadores" o capas de bajo rango a
      ciertas partes del modelo (en tu caso, a 8 capas, según se define en lora_config).

   3. Entrenar solo los adaptadores: Solo se entrenan los parámetros de estos nuevos y pequeños adaptadores. El número 163,840 es la suma
      de todos los parámetros de estas nuevas capas LoRA que se han añadido.

  En resumen:

   * 270 Millones: Es el tamaño total del modelo base, que permanece sin cambios.
   * 163,840: Es el número de parámetros nuevos y adicionales que estás entrenando. Estos parámetros son los que "aprenden" la nueva
     tarea (en este caso, generar consultas SQL) y adaptan el conocimiento del modelo original.

  Esta es la gran ventaja de LoRA: te permite especializar un modelo enorme en una tarea específica de forma muy rápida y con muchos
  menos recursos computacionales, ya que solo necesitas entrenar una fracción minúscula (<0.1%) de los parámetros totales.

In [10]:
model.freeze()
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])
num_train_params = sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
print(f"Number of trainable parameters: {num_train_params}")
model.train()

Number of trainable parameters: 163840


Model(
  (model): Gemma3Model(
    (embed_tokens): Embedding(262144, 640)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (q_proj): Linear(input_dims=640, output_dims=1024, bias=False)
        (k_proj): Linear(input_dims=640, output_dims=256, bias=False)
        (v_proj): Linear(input_dims=640, output_dims=256, bias=False)
        (o_proj): Linear(input_dims=1024, output_dims=640, bias=False)
        (q_norm): RMSNorm()
        (k_norm): RMSNorm()
        (rope): RoPE(256, traditional=False)
      )
      (mlp): MLP(
        (gate_proj): Linear(input_dims=640, output_dims=2048, bias=False)
        (down_proj): Linear(input_dims=2048, output_dims=640, bias=False)
        (up_proj): Linear(input_dims=640, output_dims=2048, bias=False)
      )
      (input_layernorm): RMSNorm()
      (post_attention_layernorm): RMSNorm()
      (pre_feedforward_layernorm): RMSNorm()
      (post_feedforward_layernorm): RMSNorm()
    )
    (layers.1): TransformerBlock(
      (self_att

In [11]:
class Metrics:
    def __init__(self) -> None:
        self.train_losses: List[Tuple[int, float]] = []
        self.val_losses: List[Tuple[int, float]] = []

    def on_train_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.train_losses.append((info["iteration"], info["train_loss"]))

    def on_val_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.val_losses.append((info["iteration"], info["val_loss"]))

In [12]:
metrics = Metrics()

# load data

In [13]:
from mlx_lm.tuner.datasets import load_hf_dataset
config = { }
train_set, val_set, test_set = load_hf_dataset(
    data_id="mlx-community/wikisql",
    tokenizer=tokenizer,
    config=config,
)

README.md:   0%|          | 0.00/841 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/74.0k [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
print(f"Test set size: {len(test_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Training set size: {len(train_set)}")
print(f"test set: {test_set[:2]}")

Test set size: 100
Validation set size: 100
Training set size: 1000
test set: {'text': ["table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What is terrence ross' nationality\nA: SELECT Nationality FROM 1-10015132-16 WHERE Player = 'Terrence Ross'", "table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What clu was in toronto 1995-96\nA: SELECT School/Club Team FROM 1-10015132-16 WHERE Years in Toronto = '1995-96'"]}


In [15]:
from mlx_lm.tuner.datasets import CacheDataset

train_dataset = CacheDataset(train_set)
val_dataset = CacheDataset(val_set)

train(
    model,
    optim.Adam(learning_rate=1e-5),
    train_dataset,
    val_dataset,
    args=training_args,
    training_callback=metrics
)


Starting training..., iters: 200


Calculating loss...: 100%|██████████| 25/25 [00:03<00:00,  7.12it/s]

Iter 1: Val loss 4.059, Val took 3.522s


Iter 10: Train loss 3.729, Learning Rate 1.000e-05, It/sec 1.574, Tokens/sec 656.535, Trained Tokens 4170, Peak mem 2.465 GB
Iter 20: Train loss 3.382, Learning Rate 1.000e-05, It/sec 2.909, Tokens/sec 1060.948, Trained Tokens 7817, Peak mem 2.465 GB
Iter 30: Train loss 2.931, Learning Rate 1.000e-05, It/sec 3.001, Tokens/sec 1093.235, Trained Tokens 11460, Peak mem 2.465 GB
Iter 40: Train loss 2.766, Learning Rate 1.000e-05, It/sec 3.117, Tokens/sec 1122.034, Trained Tokens 15060, Peak mem 2.465 GB


Calculating loss...: 100%|██████████| 25/25 [00:03<00:00,  7.13it/s]

...

Iter 150: Val loss 2.052, Val took 3.542s


Iter 150: Train loss 1.921, Learning Rate 1.000e-05, It/sec 3.085, Tokens/sec 1100.768, Trained Tokens 55650, Peak mem 2.506 GB
Iter 160: Train loss 1.950, Learning Rate 1.000e-05, It/sec 2.607, Tokens/sec 973.295, Trained Tokens 59384, Peak mem 2.506 GB
Iter 170: Train loss 1.910, Learning Rate 1.000e-05, It/sec 3.009, Tokens/sec 1074.775, Trained Tokens 62956, Peak mem 2.506 GB
Iter 180: Train loss 1.910, Learning Rate 1.000e-05, It/sec 2.932, Tokens/sec 969.710, Trained Tokens 66263, Peak mem 2.506 GB
Iter 190: Train loss 1.930, Learning Rate 1.000e-05, It/sec 2.702, Tokens/sec 1055.704, Trained Tokens 70170, Peak mem 2.506 GB


Calculating loss...: 100%|██████████| 25/25 [00:05<00:00,  4.93it/s]

Iter 200: Val loss 1.979, Val took 5.112s


Iter 200: Train loss 1.890, Learning Rate 1.000e-05, It/sec 2.790, Tokens/sec 1038.301, Trained Tokens 73891, Peak mem 2.506 GB
Iter 200: Saved adapter weights to adapters_gemma3/adapters.safetensors and adapters_gemma3/0000200_adapters.safetensors.
Saved final weights to adapters_gemma3/adapters.safetensors.


## Fusionar modelo base con adaptador

In [3]:
! python -m mlx_lm fuse  --model ./models_cache/hub/models--google--gemma-3-270m-it/snapshots/ac82b4e820549b854eebf28ce6dedaf9fdfa17b3 --adapter-path ./adapters_gemma3 --save-path ./new_gemma3 

Loading pretrained model


In [4]:
! pip install huggingface_hub
! pip install ipywidgets

# Subir modelo a HF
Utilizando el API de HF se sube el modelo a deimagjas/Phi-3.5-mini-instruct-4bit-sft

In [7]:
from huggingface_hub import create_repo, upload_folder

repo_id = "deimagjas/gemma-3-270m-it-sft"

upload_folder(
    folder_path="./new_gemma3",
    repo_id=repo_id
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...arning/new_gemma3/model.safetensors:   3%|3         | 27.9MB /  872MB            

  ...elearning/new_gemma3/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

CommitInfo(commit_url='https://huggingface.co/deimagjas/gemma-3-270m-it-sft/commit/30f8838ef60c3b7f0d0ae40e7aef85ff6aefb1d7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='30f8838ef60c3b7f0d0ae40e7aef85ff6aefb1d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/deimagjas/gemma-3-270m-it-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='deimagjas/gemma-3-270m-it-sft'), pr_revision=None, pr_num=None)

## Test HF model

In [10]:
model_path = "deimagjas/gemma-3-270m-it-sft"
model_sft, tokenizer_sft = load(model_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/872M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

In [12]:
prompt = "generate an SQL query to find all users who registered in the last 30 days"
messages = [{"role": "user", "content": prompt}]
prompt = tokenizer_sft.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate(model_sft, tokenizer_sft, prompt=prompt, verbose=True)

```sql
SELECT u.user_id, COUNT(m.registration_date) AS last_30_days_registered
FROM users u
JOIN registered_users m ON u.user_id = m.user_id
WHERE last_30_days_registered >= (SELECT last_30_days_registered FROM registered_users WHERE registered_date = '2023-12-31') ;
```
<end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn><end_of_turn>शी<end_of_turn><end_of_turn>शी<end_of_turn><end_of_

# Conclusión

El fine tunning en este caso fallo, el modelo presenta fallas en la inferencia. ¿por qué?